In [87]:
import copy, sys
import logging
logger = logging.getLogger()
logging.getLogger().handlers[:] = []
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(levelname)s: %(message)s',
    force=True
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [88]:
def show_grid(grid):
    res = ""
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] == 0:
                res += "| "
            elif grid[y][x] < 0:
                res += "|#"
            else:
                res += "|" + str(grid[y][x])
        res += "|\n"
    print(res)

In [89]:
def show_plans(plans, original_grid):
    for time in range(len(plans[0])):
        grid = copy.deepcopy(original_grid)
        for i, plan in enumerate(plans):
            x, y = plan[time]
            if grid[x][y] < 0:
                logger.error(f"Hit obstacle in plan {i} at time {time}")
            grid[x][y] = i + 1  # Mark the grid with plan index + 1 to differentiate
        show_grid(grid)

In [90]:
def get_wait_time(current_time, trajectory, plan_length):
    num_actions_left = len(set(trajectory[current_time:plan_length]))
    # No more moves after current_time, only wait
    if num_actions_left < 2:
        return 0
    # Compute number of waiting steps
    m = 0 
    for t in range(current_time+1, plan_length):
        if trajectory[t] == trajectory[t-1]:
            m += 1
    return m

def get_flex(trajectories):
    plan_length = len(trajectories[0])
    num_agents = len(trajectories)
    flexibility = []
    for agent in range(num_agents):
        flexibility.append([0 for _ in range(plan_length)])
        # Last point in plan (plan_length-1) does not have any flexibility left
        for timeVar in range(plan_length-2, -1, -1):
            m = get_wait_time(timeVar, trajectories[agent], plan_length)
            conflict = False
            for other_agent in range(num_agents):
                if agent != other_agent:
                    if trajectories[agent][timeVar] in trajectories[other_agent][timeVar+1:plan_length]:
                        index = timeVar + 1 + trajectories[other_agent][timeVar+1:plan_length].index(trajectories[agent][timeVar])
                        logger.info(f"Agent {agent} at time {timeVar} has initial wait time {m} and other agent {other_agent} crosses node {trajectories[agent][timeVar]} at time {index}")
                        # m was initial flexibility of wait time, but is reduced by the time until other agent reaches this node
                        m = m - (index - timeVar)
                        conflict = True
            logger.info(f"Flexibility of agent {agent} at time {timeVar} is {m} and flexibility at next time step is {flexibility[agent][timeVar+1] if timeVar < plan_length-2 else m}")
            if conflict and m == 0:
                flexibility[agent][timeVar] = -1
            elif timeVar == plan_length - 1:
                logger.warning(f"Agent {agent} at time {timeVar} is at end of plan so has no flexibility")
                flexibility[agent][timeVar] = m
            elif flexibility[agent][timeVar+1] < 0:
                logger.warning(f"Agent {agent} at next time {timeVar+1} has a conflict, so cannot have flexibility at current ime {timeVar}")
                # Conflict detected afterward
                flexibility[agent][timeVar] = -1
            else:
                flexibility[agent][timeVar] = max(m, flexibility[agent][timeVar+1])
    flexibility = [[x if x >= 0 else 0 for x in f] for f in flexibility]
    return flexibility

In [91]:
def print_flexibility(trajectories):
    plan_length = len(trajectories[0])
    for i, t in enumerate(trajectories):
        logger.info(f"Agent {i} has initial trajectory {t}")
    flex = get_flex(trajectories)
    logger.info(f"Flexibility: {flex}")
    for i, agent_flex in enumerate(flex):
        for t, f in enumerate(agent_flex):
            if f > 0:
                new_trajectory = trajectories[i][:t] + [trajectories[i][t]] * f + trajectories[i][t:plan_length - f]
                logger.info(f"Agent {i} at time {t} can delay up to {f} time steps")
                for j in range(len(trajectories)):
                    if i == j:
                        logger.info(f"  New trajectory agent {j}: {new_trajectory}")
                    else:
                        logger.info(f"  Old trajectory agent {j}: {trajectories[j]}")
                        for time in range(plan_length):
                            if trajectories[j][time] == new_trajectory[time]:
                                logger.error(f"Agents {i} and {j} both occupy location {trajectories[j][time]} at time {time}")
    return flex

In [92]:
# Scenario 1 - grid size 4
logger.setLevel(logging.ERROR)
trajectories = [
    [(0, 0), (1, 0), (2, 0), (3, 0), (3, 0)],
    [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)],
    [(0, 2), (1, 2), (2, 2), (2, 1), (2, 1)]
]
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0],
]
if flexibility != expected_flexibility:
    logger.error(f"Expected: {expected_flexibility}")
    logger.error(f"Received: {flexibility}")
    logger.error("ERROR: Flexibility is incorrect")

In [93]:
# Scenario 2 - corridor
logger.setLevel(logging.WARN)
grid = [
    [0, -1, -1, 0],
    [0, 0, 0, 0],
    [0, -1, -1, 0],
]
plans = [
    [(0, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 1)],
    [(1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0)]   
]
# show_plans(plans, grid)
flexibility = print_flexibility(plans)
expected_flexibility = [
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0]
]
if flexibility != expected_flexibility:
    logger.error(f"Expected: {expected_flexibility}")
    logger.error(f"Received: {flexibility}")
    logger.error("ERROR: Flexibility is incorrect")

ERROR: Expected: [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
ERROR: Received: [[3, 2, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
ERROR: ERROR: Flexibility is incorrect


In [94]:
# Scenario paper
logger.setLevel(logging.INFO)
grid = [
    [0, -1, -1, -1, -1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, -1, -1, -1, -1, -1, 0],
]
# show_grid(grid)
trajectories = [
    [(0, 0), (0, 0), (0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
    [(2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0)]
]
# show_plans(trajectories, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0 for _ in range(len(trajectories[0]))],
    [4 for _ in range(18)] + [0 for _ in range(4)]
]
if flexibility != expected_flexibility:
    logger.error(f"Expected: {expected_flexibility}")
    logger.error(f"Received: {flexibility}")
    logger.error("ERROR: Flexibility is incorrect")

INFO: Agent 0 has initial trajectory [(0, 0), (0, 0), (0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)]
INFO: Agent 1 has initial trajectory [(2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0)]
INFO: Flexibility of agent 0 at time 21 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 20 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 19 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 18 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 17 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 16 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 15 is 0 and fle

In [95]:
# Scenario three agents same node - grid size 4
trajectories = [
    [(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)],
    [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)],
    [(0, 2), (1, 2), (1, 2), (1, 1), (2, 1)]
]
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0],
]
if flexibility != expected_flexibility:
    logger.error(f"Expected: {expected_flexibility}")
    logger.error(f"Received: {flexibility}")
    logger.error("ERROR: Flexibility is incorrect")

INFO: Agent 0 has initial trajectory [(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)]
INFO: Agent 1 has initial trajectory [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)]
INFO: Agent 2 has initial trajectory [(0, 2), (1, 2), (1, 2), (1, 1), (2, 1)]
INFO: Flexibility of agent 0 at time 3 is 0 and flexibility at next time step is 0
INFO: Agent 0 at time 2 has initial wait time 0 and other agent 2 crosses node (1, 1) at time 3
INFO: Flexibility of agent 0 at time 2 is -1 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 1 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 0 at time 0 is 0 and flexibility at next time step is 0
INFO: Flexibility of agent 1 at time 3 is 0 and flexibility at next time step is 0
INFO: Agent 1 at time 2 has initial wait time 1 and other agent 2 crosses node (2, 1) at time 4
INFO: Flexibility of agent 1 at time 2 is -1 and flexibility at next time step is 0
INFO: Agent 1 at time 1 has initial wait time 1 and other agent 0 crosses 

In [96]:
# Scenario double corridor